In [6]:
from sqlalchemy import create_engine
import pandas as pd

In [7]:
# 1. Establish connection to the database
def create_db_engine(user, password, host, db_name):
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db_name}')
    return engine

In [8]:
# 2. Function to get rental data for a specific month and year
def rentals_month(engine, month, year):
    query = f"""
    SELECT r.rental_id, r.rental_date, r.inventory_id, r.customer_id, r.return_date
    FROM rental r
    WHERE MONTH(r.rental_date) = {month} AND YEAR(r.rental_date) = {year};
    """
    df = pd.read_sql(query, engine)
    return df

In [9]:
# 3. Function to count the rentals made by each customer
def rental_count_month(df, month, year):
    rental_counts = df.groupby('customer_id').size().reset_index(name='rental_count')
    rental_counts[f'rentals_{month:02d}_{year}'] = rental_counts['rental_count']
    rental_counts.drop(columns=['rental_count'], inplace=True)
    return rental_counts

In [10]:
# 4. Function to compare the rental counts for two different months
def compare_rentals(df1, df2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer')
    merged_df.fillna(0, inplace=True)
    merged_df['difference'] = merged_df.iloc[:, 1] - merged_df.iloc[:, 2]
    return merged_df